In [1]:
import pandas as pd
import mibian as mb
import yfinance as yf
from datetime import datetime

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Black Scholes Assumptions

In [2]:
## Assume volatility is 20
vol = 20

## Assume free interest rate is 3%
interest_rate = 3

## Yahoo Finance Ticker Object

In [3]:
stock_ticker = "CCL"
ticker = yf.Ticker(stock_ticker)

## Option Chain Snapshot Helper Function

In [4]:
# Helper Functions for Constant Fields
def _constant_fields(df, ticker, expiration_date, interest_rate, stock_ticker):
    df['underlyingPrice'] = ticker.history(period="0d").iloc[0]['Close']
    df['ticker'] = stock_ticker
    df['interestRate'] = interest_rate
    df['expirationDate'] = datetime.strptime(expiration_date, '%Y-%m-%d')
    df['daysToExpiration'] = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days + 1
    return df

## Concatenate All Available Option Chain Snapshots in DataFrame

In [5]:
# Calls DF
call_chains_df = pd.DataFrame()
# Puts DF
put_chains_df = pd.DataFrame()

# Iterate over all available expiration dates
for expiration_date in ticker.options:
    
    # Calls DF
    calls_df = _constant_fields(ticker.option_chain(expiration_date).calls, ticker, expiration_date, interest_rate, stock_ticker)
    call_chains_df = call_chains_df.append(calls_df, ignore_index=True)
    
    # Puts DF
    puts_df = _constant_fields(ticker.option_chain(expiration_date).puts, ticker, expiration_date, interest_rate, stock_ticker)
    put_chains_df = put_chains_df.append(puts_df, ignore_index=True)

## Black Scholes Helper Functions

In [6]:
def black_scholes_vol(underlyingPrice, strike, interestRate, daysToExpiration, vol):
    return mb.BS([underlyingPrice, strike, interestRate, daysToExpiration], volatility=vol)

def black_scholes_call_output(mb_object):
    return mb_object.callPrice, mb_object.callDelta, mb_object.callDelta2, mb_object.callTheta, mb_object.callRho, mb_object.vega, mb_object.gamma

def black_scholes_put_output(mb_object):
    return mb_object.putPrice, mb_object.putDelta, mb_object.putDelta2, mb_object.putTheta, mb_object.putRho, mb_object.vega, mb_object.gamma

## Generate Black Scholes Objects and Values

In [7]:
call_chains_df['blackScholes'] = call_chains_df.apply(lambda x: black_scholes_vol(x['underlyingPrice'], x['strike'], x['interestRate'], x['daysToExpiration'], vol), axis=1)
call_chains_df['blackScholesCallPrice'], call_chains_df['blackScholesCallDelta'], call_chains_df['blackScholesCallDualDelta'], call_chains_df['blackScholesCallTheta'], call_chains_df['blackScholesCallRho'], call_chains_df['blackScholesVega'], call_chains_df['blackScholesGamma'] = \
    zip(*call_chains_df['blackScholes'].map(lambda x: black_scholes_call_output(x)))


put_chains_df['blackScholes'] = put_chains_df.apply(lambda x: black_scholes_vol(x['underlyingPrice'], x['strike'], x['interestRate'], x['daysToExpiration'], vol), axis=1)
put_chains_df['blackScholesPutPrice'], put_chains_df['blackScholesPutDelta'], put_chains_df['blackScholesPutDualDelta'], put_chains_df['blackScholesPutTheta'], put_chains_df['blackScholesPutRho'], put_chains_df['blackScholesVega'], put_chains_df['blackScholesGamma'] = \
    zip(*put_chains_df['blackScholes'].map(lambda x: black_scholes_put_output(x)))

## Preview Call Output

In [8]:
call_chains_df.head(10)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,expirationDate,daysToExpiration,blackScholes,blackScholesCallPrice,blackScholesCallDelta,blackScholesCallDualDelta,blackScholesCallTheta,blackScholesCallRho,blackScholesVega,blackScholesGamma
0,CCL200626C00001000,2020-06-19 13:52:50,1.0,17.7,14.6,17.9,0.000000,0.000000,2.0,0,...,2020-06-25,3,<mibian.BS object at 0x127869d90>,16.230247,1.0,-0.999753,-0.000082,0.000082,0.000000e+00,0.000000e+00
1,CCL200626C00002000,2020-05-27 18:40:40,2.0,15.3,15.0,15.5,0.000000,0.000000,1.0,0,...,2020-06-25,3,<mibian.BS object at 0x127869ca0>,15.230493,1.0,-0.999753,-0.000164,0.000164,0.000000e+00,0.000000e+00
2,CCL200626C00008000,2020-06-02 18:39:50,8.0,8.8,9.0,10.0,0.000000,0.000000,2.0,0,...,2020-06-25,3,<mibian.BS object at 0x1278695b0>,9.231972,1.0,-0.999753,-0.000657,0.000657,0.000000e+00,0.000000e+00
3,CCL200626C00009000,2020-05-27 13:40:54,9.0,8.4,7.9,8.4,0.000000,0.000000,1.0,0,...,2020-06-25,3,<mibian.BS object at 0x127869820>,8.232219,1.0,-0.999753,-0.000740,0.000740,7.524822e-282,1.541939e-279
4,CCL200626C00009500,2020-06-11 17:35:39,9.5,8.4,7.5,7.9,0.000000,0.000000,2.0,0,...,2020-06-25,3,<mibian.BS object at 0x127869160>,7.732342,1.0,-0.999753,-0.000781,0.000781,2.281679e-237,4.675473e-235
5,CCL200626C00010000,2020-06-22 19:21:18,10.0,6.9,7.0,7.6,-2.079999,-23.162579,1.0,0,...,2020-06-25,3,<mibian.BS object at 0x127869850>,7.232465,1.0,-0.999753,-0.000822,0.000822,9.736300e-199,1.995102e-196
6,CCL200626C00010500,2020-06-05 17:58:31,10.5,11.9,6.5,7.0,0.000000,0.000000,1.0,3,...,2020-06-25,3,<mibian.BS object at 0x12783e790>,6.732589,1.0,-0.999753,-0.000863,0.000863,3.222113e-165,6.602551e-163
7,CCL200626C00011000,2020-06-16 13:49:52,11.0,9.9,6.0,6.9,0.000000,0.000000,10.0,17,...,2020-06-25,3,<mibian.BS object at 0x12783efd0>,6.232712,1.0,-0.999753,-0.000904,0.000904,3.465408e-136,7.101098e-134
8,CCL200626C00011500,2020-06-16 15:03:12,11.5,5.3,5.6,5.9,-3.200000,-37.647057,1.0,0,...,2020-06-25,3,<mibian.BS object at 0x12783e7f0>,5.732835,1.0,-0.999753,-0.000945,0.000945,4.071129e-111,8.342302e-109
9,CCL200626C00012000,2020-06-22 19:55:56,12.0,5.1,5.0,5.4,-2.160000,-29.752070,39.0,0,...,2020-06-25,3,<mibian.BS object at 0x12783e100>,5.232959,1.0,-0.999753,-0.000986,0.000986,1.467688e-89,3.007495e-87


## Preview Put Output

In [9]:
put_chains_df.head(10)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,expirationDate,daysToExpiration,blackScholes,blackScholesPutPrice,blackScholesPutDelta,blackScholesPutDualDelta,blackScholesPutTheta,blackScholesPutRho,blackScholesVega,blackScholesGamma
0,CCL200626P00004000,2020-06-19 17:50:35,4.0,0.10,0.0,0.05,0.00,0.000000,2.0,2,...,2020-06-25,3,<mibian.BS object at 0x127951b80>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
1,CCL200626P00005000,2020-06-19 17:54:05,5.0,0.01,0.0,0.05,-0.04,-79.999990,1.0,46,...,2020-06-25,3,<mibian.BS object at 0x127951ac0>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
2,CCL200626P00006000,2020-06-19 18:09:19,6.0,0.05,0.0,0.25,0.00,0.000000,1.0,0,...,2020-06-25,3,<mibian.BS object at 0x1279afa00>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
3,CCL200626P00007000,2020-06-19 17:53:35,7.0,0.40,0.0,0.05,0.00,0.000000,1.0,4,...,2020-06-25,3,<mibian.BS object at 0x1279af040>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
4,CCL200626P00008000,2020-06-19 19:13:33,8.0,0.05,0.0,0.00,0.00,0.000000,8.0,104,...,2020-06-25,3,<mibian.BS object at 0x1279afac0>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
5,CCL200626P00008500,2020-06-19 19:51:27,8.5,0.05,0.0,0.05,0.00,0.000000,5.0,18,...,2020-06-25,3,<mibian.BS object at 0x1279afaf0>,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
6,CCL200626P00009000,2020-06-22 16:57:30,9.0,0.05,0.0,0.05,0.00,0.000000,1.0,139,...,2020-06-25,3,<mibian.BS object at 0x1279afb20>,1.169530e-283,-1.343068e-281,2.572529e-281,-2.506371e-281,-1.902967e-284,7.524822e-282,1.541939e-279
7,CCL200626P00009500,2020-06-22 16:57:17,9.5,0.05,0.0,0.05,-0.05,-50.000000,4.0,162,...,2020-06-25,3,<mibian.BS object at 0x1279afb50>,4.217451e-239,-4.441385e-237,8.059709e-237,-7.599303e-237,-6.293197e-240,2.281679e-237,4.675473e-235
8,CCL200626P00010000,2020-06-22 15:21:45,10.0,0.04,0.0,0.05,-0.06,-60.000004,78.0,404,...,2020-06-25,3,<mibian.BS object at 0x1279afb80>,2.153644e-200,-2.073376e-198,3.574581e-198,-3.242495e-198,-2.938012e-201,9.736300e-199,1.995102e-196
9,CCL200626P00010500,2020-06-22 19:24:26,10.5,0.05,0.0,0.05,-0.05,-50.000000,261.0,0,...,2020-06-25,3,<mibian.BS object at 0x1279afbb0>,8.593983e-167,-7.535257e-165,1.237318e-164,-1.072970e-164,-1.067823e-167,3.222113e-165,6.602551e-163
